In [ ]:
import cv2
import mediapipe as mp
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def classifyPose(landmarks):
    left_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
    left_elbow = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
    left_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]

    right_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
    right_elbow = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
    right_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

    if (left_wrist and left_elbow and left_shoulder and right_wrist and right_elbow and right_shoulder):
        # Calculate the slope of the line passing through left wrist, elbow, and shoulder
        left_slope = (left_elbow.y - left_wrist.y) / (left_elbow.x - left_wrist.x + 1e-10)
        left_slope_shoulder = (left_shoulder.y - left_wrist.y) / (left_shoulder.x - left_wrist.x + 1e-10)

        # Calculate the slope of the line passing through right wrist, elbow, and shoulder
        right_slope = (right_elbow.y - right_wrist.y) / (right_elbow.x - right_wrist.x + 1e-10)
        right_slope_shoulder = (right_shoulder.y - right_wrist.y) / (right_shoulder.x - right_wrist.x + 1e-10)

        # Check if left hand is straight and right hand is bent and the body is inclined
        if abs(left_slope) < 0.2 and abs(left_slope_shoulder) > 1.5 and abs(right_slope) > 1.5 and abs(right_slope_shoulder) < 0.2:
            return 'Side Plank'

    return 'Unknown Pose'

pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

video = cv2.VideoCapture(0)
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_video.process(rgb_frame)

    if results.pose_landmarks:
        pose_label = classifyPose(results)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(frame, pose_label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

    cv2.imshow('Pose Detection', frame)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()
